In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [12]:
# Hyperparameters
num_epochs = 5
batch_size = 64
learning_rate = 0.001

In [13]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


In [14]:
# Define the CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.pool(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.pool(out)
        out = out.view(out.size(0), -1)  # Flatten
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out

model = CNN()


In [15]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [16]:
# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/5], Step [100/938], Loss: 0.0788
Epoch [1/5], Step [200/938], Loss: 0.1055
Epoch [1/5], Step [300/938], Loss: 0.0274
Epoch [1/5], Step [400/938], Loss: 0.0231
Epoch [1/5], Step [500/938], Loss: 0.1567
Epoch [1/5], Step [600/938], Loss: 0.0735
Epoch [1/5], Step [700/938], Loss: 0.0750
Epoch [1/5], Step [800/938], Loss: 0.1269
Epoch [1/5], Step [900/938], Loss: 0.0276
Epoch [2/5], Step [100/938], Loss: 0.0121
Epoch [2/5], Step [200/938], Loss: 0.0248
Epoch [2/5], Step [300/938], Loss: 0.0326
Epoch [2/5], Step [400/938], Loss: 0.0346
Epoch [2/5], Step [500/938], Loss: 0.0327
Epoch [2/5], Step [600/938], Loss: 0.0158
Epoch [2/5], Step [700/938], Loss: 0.0011
Epoch [2/5], Step [800/938], Loss: 0.0211
Epoch [2/5], Step [900/938], Loss: 0.0154
Epoch [3/5], Step [100/938], Loss: 0.0165
Epoch [3/5], Step [200/938], Loss: 0.0032
Epoch [3/5], Step [300/938], Loss: 0.0638
Epoch [3/5], Step [400/938], Loss: 0.0215
Epoch [3/5], Step [500/938], Loss: 0.0071
Epoch [3/5], Step [600/938], Loss:

In [17]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the model on the 10000 test images: 99.24%


In [18]:
# Save the trained model
torch.save(model.state_dict(), 'mnist_model.pth')